In [1]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

apipath = r'H:\\My Drive\\config\\hbqa.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']

In [3]:
df_hbqa_auto= pd.read_csv(datapath+'06.1-HBQA_Auto.csv')
df_hbqa_auto.head(4)

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer,WordsInQues,WordsInAns
0,0,Book01_002,1,Who asked Sauti about his journey?,One of the Rishis beginning the conversation a...,6,12
1,1,Book01_002,1,What did the Rishis wish to hear from Sauti?,The Rishis wished to hear the wonderful narrat...,9,10
2,2,Book01_002,1,What did the Rishis ask Sauti to recite?,The Rishis asked Sauti to recite the sacred st...,8,32
3,3,Book01_002,1,What is the Mahabharata?,The Mahabharata is a sacred work composed in e...,4,29


### Preparing 06-HBQA_Manual for Merging with 06-HBQA_Auto

In [4]:
df_hbqa_manual=pd.read_csv(r'D:\github\HBQA\DBA\Data\\06.2-HBQA_Manual.csv')
df_hbqa_manual.head(4)

,QuesId,SectionId,ContextId,Question,Answer,Reference
0,1,Book01_001,1,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...,"Reference: ""Sauti said, 'Having heard the dive..."
1,2,Book01_001,1,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n...","Reference: ""The Purana, first promulgated by t..."
2,3,Book01_001,1,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg...","Reference: ""In this world, when it was destitu..."
3,4,Book01_001,2,How will the universe be affected at the end o...,At the end of the world and the expiration of ...,"Reference: ""And what is seen in the universe, ..."


#### <font color=red> Run just below cell only once</font>

In [5]:
# Renaming columns
df_hbqa_manual.rename(columns= {"QuesId":"Ques_Id","SectionId":"Section_Id","ContextId":"Chunk_Id","Answer":"Ref_Answer"}, inplace=True)

# Drop Column
df_hbqa_manual.drop(['Reference'], axis=1, inplace=True)

In [7]:
df_hbqa_manual.head(3)

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer
0,1,Book01_001,1,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...
1,2,Book01_001,1,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n..."
2,3,Book01_001,1,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg..."


#### <font color=red> Run just below cell only once</font>

In [8]:
df_hbqa_manual['Chunk_Id']=df_hbqa_manual['Chunk_Id']+10000
df_hbqa_manual['Ques_Id']=df_hbqa_manual['Ques_Id']+10000

In [9]:
df_hbqa_manual.head(3)

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer
0,10001,Book01_001,10001,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...
1,10002,Book01_001,10001,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n..."
2,10003,Book01_001,10001,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg..."


In [10]:
df_hbqa_manual['WordsInQues'] = df_hbqa_manual.Question.str.split(' ').apply(len)
df_hbqa_manual['WordsInAns']  = df_hbqa_manual.Ref_Answer.str.split(' ').apply(len)
df_hbqa_manual.head(3)

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer,WordsInQues,WordsInAns
0,10001,Book01_001,10001,Who composed the Mahabharata and recited it at...,The Mahabharata was composed by Krishna-Dwaipa...,13,17
1,10002,Book01_001,10001,What is the nature of the Mahabharata and what...,"The Mahabharata is a sacred work, an eminent n...",12,43
2,10003,Book01_001,10001,How did creation begin according to the Mahabh...,"At the beginning of the Yuga, from a mighty eg...",8,47


### Merge Auto and Manual Created QA

In [11]:
df_hbqa = pd.concat([df_hbqa_manual,df_hbqa_auto])
print(df_hbqa.shape)
df_hbqa.sample(5)

(2204, 7)


,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer,WordsInQues,WordsInAns
699,10700,Book01_129,10517,Whom did Bhima meet first after arriving on th...,Bhima first met his mother and his eldest brot...,13,17
1062,11063,Book01_221,10829,How did Krishna react to Arjuna's infatuation ...,Krishna observed Arjuna contemplating Bhadra w...,9,48
552,10553,Book01_100,10414,What vow did the son of Ganga adopt after reli...,The son of Ganga adopted the vow of Brahmachar...,15,21
230,230,Book01_059,47,What did Astika tell Vasuki?,Astika told Vasuki that he would strive earnes...,5,23
1013,11014,Book01_210,10797,How did the Pandavas pass their days after obt...,The Pandavas passed their days in joy and happ...,11,13


In [12]:
df_hbqa.to_csv(datapath+'06.3-HBQA_Combined.csv')

### Check Duplicate Questions

In [13]:
df_hbqa_unique = pd.DataFrame(df_hbqa.groupby('Question')['Ques_Id'].count().sort_values(ascending=False)).reset_index()
df_hbqa_unique


,Question,Ques_Id
0,How did Kasyapa pacify the Valakhilyas?,6
1,Who was the father of Astika?,6
2,What did Takshaka ask Kasyapa to do?,5
3,What did Sesha do after hearing of the curse?,5
4,Who offered his virtuous daughter Uttara to Ar...,5
...,...,...
1665,What did Parikshit do to express his anger tow...,1
1666,What did Parikshit do to Jaratkaru when he enc...,1
1667,What did Parikshit do to Jaratkaru that caused...,1
1668,What did Parikshit do that caused him to be cu...,1


In [14]:
df_hbqa_unique.to_csv(datapath+'07.1-HBQA_UniqueQuesOnly.csv')

In [15]:
df_hbqa_unique.loc[ df_hbqa_unique.Ques_Id>1,: ].to_csv(datapath+'07.2-HBQA_DuplicateQuesOnly.csv')

In [16]:
df_hbqa_unique.loc[ df_hbqa_unique.Ques_Id==1,: ].to_csv(datapath+'07.3-HBQA_NonDuplicatedQuesOnly.csv')

#### A Question Duplicated How Many Time?

In [17]:
check_question = df_hbqa_unique.loc[0,'Question']
df_hbqa.loc[df_hbqa.Question==check_question,:]

,Ques_Id,Section_Id,Chunk_Id,Question,Ref_Answer,WordsInQues,WordsInAns
152,152,Book01_035,32,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...,6,29
383,383,Book01_093,79,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...,6,29
560,560,Book01_153,129,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...,6,28
686,686,Book01_220,179,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...,6,29
854,854,Book03_085,329,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...,6,29
1020,1020,Book03_199,429,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...,6,29


### Preparing 04-Chunk_with_Prompts_for_Manual for Merging with 04-Chunk_with_Prompts

In [18]:
df_chunk_for_manul = pd.read_csv(datapath+'03-Chunked-Book_for_Manual.csv')
print(df_chunk_for_manul.shape)
df_chunk_for_manul.head(3)

(7315, 5)


,SetChunk_Len,Section_Id,Chunk_Id,Chunk,Chunk_Letters
0,6000,Book01_001,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,5985
1,6000,Book01_001,2,"And what is seen in the universe, whether anim...",5674
2,6000,Book01_001,3,"""Ganesa, upon hearing this address, thus answe...",5169


In [19]:
df_chunk_for_manul.drop(['SetChunk_Len'], axis=1, inplace=True)

<font color=red>Run below cell only once</font>

In [20]:
df_chunk_for_manul['Chunk_Id'] = df_chunk_for_manul['Chunk_Id'] + 10000

In [21]:
df_chunk_for_manul['Chunk_Words'] = df_chunk_for_manul.Chunk.str.split(' ').apply(len)
df_chunk_for_manul['Chunk_Approx_Tokens'] = (df_chunk_for_manul['Chunk_Words']/4).round(0)

In [22]:
df_chunk_for_manul.sample(4)

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens
2024,Book03_304,12072,"Vaisampayana continued, ""Thereupon the princes...",1818,301,75.0
1212,Book03_041,11260,"""After these words had been spoken, the Kuru p...",1514,224,56.0
310,Book01_086,10358,"Section LXXXVI\n(Sambhava Parva continued)\n""V...",1743,268,67.0
6726,Book13_143,16774,"Sudra, if he is established on good conduct, i...",2000,330,82.0


### Merged Chunk Dataset

In [23]:
df_chunk = pd.read_csv(datapath+'03-Chunked_Book.csv')
print(df_chunk.shape)
df_chunk.head(3)

(1963, 7)


,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Approx_Ques
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,7511,1214,1615.0,4
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,7551,1261,1677.0,5
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",6986,1162,1545.0,4


In [24]:
df_chunk.drop(['Approx_Ques'], axis=1, inplace=True)

In [25]:
df_chunk_final = pd.concat([df_chunk,df_chunk_for_manul])

In [26]:
df_chunk_final

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,7511,1214,1615.0
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,7551,1261,1677.0
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",6986,1162,1545.0
3,Book01_002,5,'When I heard that Yudhishthira had been follo...,10832,1949,2592.0
4,Book01_002,6,"'Alas! Gandhari, destitute of children, grand-...",7755,1246,1657.0
...,...,...,...,...,...,...
7310,Book18_006,17358,"""During the recitation of the Bhishma-parva, O...",1857,253,63.0
7311,Book18_006,17359,"""During the Mahaprasthanika, similar presents ...",551,65,16.0
7312,Book18_006,17360,"""Upon the conclusion of all the Parvas, the ho...",2238,345,86.0
7313,Book18_006,17361,"""I have thus indicated the ordinances, O forem...",1220,187,47.0


In [ ]:
df_chunk_final.to_csv(datapath + '03-Chunked-Book-Combined.csv')

### Create HBQA Final Dataset
Get Quest_Id, Section_Id, Chunk_Id, Chunk, Question, Answer of these unique questions.

In [58]:
df_hbqa.loc[df_hbqa['Question']==ques,:].head(1)['Section_Id'].values


array(['Book01_035'], dtype=object)

In [77]:
df_mahabharat = pd.DataFrame(columns=['Ques_Id','Chunk_Id','Section_Id','Question','Ref_Answer'])
df_mahabharat

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer


In [83]:
df_chunk_final

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,7511,1214,1615.0
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,7551,1261,1677.0
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",6986,1162,1545.0
3,Book01_002,5,'When I heard that Yudhishthira had been follo...,10832,1949,2592.0
4,Book01_002,6,"'Alas! Gandhari, destitute of children, grand-...",7755,1246,1657.0
...,...,...,...,...,...,...
7310,Book18_006,17358,"""During the recitation of the Bhishma-parva, O...",1857,253,63.0
7311,Book18_006,17359,"""During the Mahaprasthanika, similar presents ...",551,65,16.0
7312,Book18_006,17360,"""Upon the conclusion of all the Parvas, the ho...",2238,345,86.0
7313,Book18_006,17361,"""I have thus indicated the ordinances, O forem...",1220,187,47.0


In [90]:
chunk_id

32

In [105]:
df_chunk_final[df_chunk_final['Chunk_Id']==chunk_id]

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens


In [110]:
df_chunk_final[df_chunk_final['Chunk_Id']==32] #['Chunk']

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens


In [100]:

df_chunk_final.loc[df_chunk_final['Chunk_Id']==str(chunk_id) ,:]#.head(1)

,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens


In [78]:
rowid=0
for row in range(df_hbqa_unique.shape[0]):
    ques= df_hbqa_unique.loc[row,'Question']
    #print(df_hbqa.loc[df_hbqa['Question']==ques,:])

    temp = df_hbqa.loc[df_hbqa['Question']==ques,:].head(1)
    chunk_id = temp['Chunk_Id'].values[0]
    section_id = temp['Section_Id'].values[0]
    ques_id = temp['Ques_Id'].values[0]
    ref_answer = temp['Ref_Answer'].values[0]
    chunk =     df_chunk_final['Chunk']
    row = (ques_id, chunk_id, section_id, ques,ref_answer)
    df_mahabharat.loc[rowid] = row
    rowid+=1
    
    break

In [79]:
df_mahabharat

,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer
0,152,32,Book01_035,How did Kasyapa pacify the Valakhilyas?,Kasyapa pacified the Valakhilyas by telling th...


In [ ]:
# Finally save 
# df_hbqa_unique.to_csv(datapath+'07.1-HBQA_UniqueQuesOnly.csv')